# Fine-Tuning Model For Metaphor Detection: Local Model PEFT.

In this notebook, I will walk you through the process how local model is fine-tuned to detect metaphor in our study.

This includes model fine-tuning via parameter efficient fine-tuning, and then model inferring.

First, let us start with the dependencies. These packages are needed: transformers peft pandas datasets torch

In [ ]:
!pip install transformers peft pandas datasets

Import packages

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments,Trainer
from peft import LoraConfig, TaskType, get_peft_model
import pandas as pd
from datasets import Dataset
import torch
import re

Before moving into fine-tuning, you'll need to load the dataset.

In [ ]:
data_fp="data/metaphor_dataset.csv"
data_df=pd.read_csv(data_fp)

And then, perform the train-test split.

This would split the dataset into two independent parts: train set and test set.

The train set will be exposed to the model in fine-tuning, which would make the model "learn" the traits of metaphor.

The test set will remain unexposed to the model during fine-tuning. It will be reserved for evaluation on the performance of model.

Here, we use a typical train-test split ratio of 8:2, and to maximize replicability, you may also wish to set a random seed(here seed =1).

In [ ]:
seed=1
train_ratio=0.8

In [ ]:
train_df=data_df.sample(frac=train_ratio,random_state=seed)
test_df=data_df.drop(index=train_df.index)

Load model and tokenizer from huggingface.

Please note: some model used in this study is gated. You will need to apply for access. You apply for access is free, and it would typically get approved within 72 hours. Please check your huggingface account for details.

In [ ]:
tknzid="meta-llama/Llama-3.2-1B-Instruct"
modelid="meta-llama/Llama-3.2-1B-Instruct"

tokenizer=AutoTokenizer.from_pretrained(tknzid)
model=AutoModelForCausalLM.from_pretrained(modelid,torch_dtype=torch.float16,device_map="cuda") # note please refer to your own spec: cuda is for nvidia device.

tokenizer.pad_token=tokenizer.eos_token
model.pad_token=tokenizer.eos_token

Wrap the text in train set in chat

In [ ]:
user_msg_0="Can you please identify and tag the metaphors in the following text? "

messages_lists=[]
for idx in range(0,train_df.shape[0]):
    text=train_df.iloc[idx]["u"].replace("\n"," ")
    raw_text=train_df.iloc[idx]["plain"].replace("\n"," ")
    messages=[
            {"role":"user","content":user_msg_0+"\n"+raw_text},
            {"role":"assistant","content":text},
        ]
    messages_lists.append(messages)



Apply chat template and tokenize the texts for fine-tuning.

In [ ]:
ds=Dataset.from_dict({"text":messages_lists})

def tokenize_item(item):
    item=tokenizer.apply_chat_template(item["text"],tokenize=False)
    item=tokenizer(item,padding=True,truncation=True)
    item["labels"]=item["input_ids"].copy()
    return item

ds=ds.map(tokenize_item,batched=True)

Setup parameter efficient fine-tuning.

In [ ]:
peft_config = LoraConfig(
     target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    task_type=TaskType.CAUSAL_LM,
)
model=get_peft_model(model, peft_config)

training_args = TrainingArguments(
    output_dir="chatbot",
    per_device_train_batch_size=1,
    eval_strategy="no",
    report_to="none",
    fp16=True,
)

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=ds)

Run the following to start fine-tuning

In [ ]:
trainer.train()

Don't forget to save your model after fine-tuning.

In [ ]:
trainer.save_model("ft_local_llama1b")

Good. Till now we've fine-tuned the model. The next step: inferring. Let's try our model on some new text(or text in test set).

In [ ]:
raw_text=test_df.iloc[idx]["plain"].replace("\n"," ")

Infer on fine-tuned model

In [ ]:
user_msg_0="Can you please identify and tag the metaphors in the following text?"

messages=[
        {"role":"user","content":user_msg_0+"\n"+raw_text},
    ]

ct=tokenizer.apply_chat_template(messages,tokenize=False)
ct=tokenizer(ct,return_tensors="pt",truncation=True)

outputs=model.generate(input_ids=ct["input_ids"].to("cuda"),attention_mask=ct["attention_mask"].to("cuda"))

out_text=tokenizer.decode(outputs[0])
pttn="<\\|start_header_id\\|>assistant<\\|end_header_id\\|>(.*)"
res=re.search(pttn,out_text,flags=re.DOTALL)
if(res==None):
    rs=""
else:
    rs=res.group(1)

View the result.

In [ ]:
print(rs)